# インポート

In [ ]:
%reload_ext autoreload

# ROOT_DIRの設定
import sys
import os
# Notebook のディレクトリ（`notebook.ipynb` がある場所）から 2 つ上のディレクトリを取得
parent_dir = os.path.abspath(os.path.join(os.getcwd(), "../.."))
# sys.path に追加
sys.path.append(parent_dir)

import pandas as pd

from common.src import scraping
from common.src import create_rawdf
from common.src import constants

# データ加工

### レース結果テーブルの加工

In [ ]:
df = pd.read_csv(constants.RACE_RESULTS_RAWDF_DIR / "race_results_2023.csv", sep="\t")
len(df)

In [ ]:
df.iloc[:, 0:10]